In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [3]:
wine_data = "Datasets/winemag-data-130k-v2.csv"

In [4]:
wine_data_df = pd.read_csv(wine_data)

In [5]:
wine_data_df.head(2)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos


In [6]:
wine_data_df.columns

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')

In [7]:
reduced_wine_data_df = wine_data_df[["country", "points", "price", "variety", "province", "winery"]]

In [8]:
drop_na_df = reduced_wine_data_df.dropna()

In [9]:
drop_na_df.count()

country     120915
points      120915
price       120915
variety     120915
province    120915
winery      120915
dtype: int64

In [116]:
#groupbys for dataset
country_group = drop_na_df.groupby("country")
province_group = drop_na_df.groupby("province")
winery_group = drop_na_df.groupby("winery")
variety_group = drop_na_df.groupby("variety")
country_by_province = drop_na_df.groupby(["country", "province"])
country_by_province.mean()
country_by_variety = drop_na_df.groupby(["country", "variety"])
country_group.mean()

,points,price
country,,
Argentina,86.710330,24.510117
Armenia,87.500000,14.500000
Australia,88.595466,35.437663
Austria,90.190782,30.762772
Bosnia and Herzegovina,86.500000,12.500000
Brazil,84.659574,23.765957
Bulgaria,87.936170,14.645390
Canada,89.377953,35.712598
Chile,86.495130,20.787316


In [107]:
us = drop_na_df[drop_na_df["country"]=="US"]
france = drop_na_df[drop_na_df["country"]=="France"]
most_produced= drop_na_df.groupby("variety").count()["points"].sort_values(ascending = False)
#pinot_noir = drop_na_df[drop_na_df["variety"]=="Pinot Noir"]
#pinot_noir['points'].value_counts()
most_produced

variety
Pinot Noir                      12785
Chardonnay                      11077
Cabernet Sauvignon               9384
Red Blend                        8466
Bordeaux-style Red Blend         5340
Riesling                         4971
Sauvignon Blanc                  4780
Syrah                            4086
Rosé                             3261
Merlot                           3061
Zinfandel                        2708
Malbec                           2593
Sangiovese                       2377
Nebbiolo                         2331
Portuguese Red                   2196
White Blend                      2167
Sparkling Blend                  2027
Tempranillo                      1788
Rhône-style Red Blend            1404
Pinot Gris                       1388
Cabernet Franc                   1304
Champagne Blend                  1211
Grüner Veltliner                 1145
Pinot Grigio                     1002
Portuguese White                  986
Viognier                          985
Gewü

In [126]:
country_production = drop_na_df["country"].value_counts()

In [150]:
#Number of bottles with score greater than 80 indexed by country in descending order
sort_production_df = pd.DataFrame({"Bottles with Score >80":country_production})
#iloc df of top 10 producers for graphing
top_10_producers = sort_production_df.iloc[0:9,:]


In [151]:
#average score in descending order
avg_points_descending=country_group.mean()["points"].sort_values(ascending=False)
sort_scores = pd.DataFrame({"Average Score":avg_points_descending})
#iloc of top 10 scorers for graphing (convert to value_counts for series if needed)
top_10_scorers = sort_scores.iloc[0:9,:]


In [157]:
#Dataframe of top 10 producers average score (optional sort by descending)
index_scorers = top_10_producers.index.to_list()
top_10_producers_scores = sort_scores.loc[index_scorers,:]
top_10_producers_scores.sort_values(by = ["Average Score"], ascending = False)

,Average Score
country,
Austria,90.190782
France,88.734867
Italy,88.618186
Australia,88.595466
US,88.566387
Portugal,88.316718
Spain,87.290735
Argentina,86.710330
Chile,86.495130


In [172]:
#highest scoring varieties
mean_by_variety = variety_group.mean()
mean_by_variety.sort_values(by=["points"], ascending = False)
#most reviewed varieties
most_pop_varieties = variety_group.count().sort_values(by = ['points'], ascending = False)
top_10_varieties = most_pop_varieties.iloc[0:9,:].index.to_list()
#top 10 number of reviews by score and price
top_10_varieties_df = mean_by_variety.loc[top_10_varieties, :].sort_values(by = ['points'], ascending = False)

,points,price
variety,,
Riesling,89.439147,32.005633
Pinot Noir,89.409230,47.532734
Syrah,89.290749,39.137788
Bordeaux-style Red Blend,88.792135,47.210861
Cabernet Sauvignon,88.610294,47.944267
Red Blend,88.379754,35.890739
Chardonnay,88.302880,34.523788
Sauvignon Blanc,87.406904,20.225523
Rosé,86.885925,18.506900


In [ ]:
#Dataframes of top 10 varieties
